In [142]:
import sqlalchemy as db


In [143]:
# sqlite://<nohostname>/<path>
# where <path> is relative:
engine = db.create_engine("sqlite:///restaurant.db")

In [144]:
connection = engine.connect()
metadata = db.MetaData()


In [145]:


ingredient_table = db.Table('Ingredient', metadata,
              db.Column('nom_ingredient', db.String(), primary_key=True),
              db.Column('prix_achat', db.Float(), default=0.0),
              )



In [146]:


pays_table = db.Table('Pays', metadata,
              db.Column('pays', db.String(), primary_key=True),
              )



In [147]:


restaurant_table = db.Table('Restaurant', metadata,
              db.Column('code_postal', db.String(), primary_key=True),
              db.Column('parking', db.Integer(),nullable=False),
              db.Column('espace_enfant', db.Integer(),default=0),
              db.Column('pmr', db.Integer(),default=0),
              db.Column('borne', db.Integer(),default=0),
              db.Column('capacite', db.Integer(),default=0),
              db.Column('pays', db.String(), db.ForeignKey("Pays.pays"),nullable=False),
              )



In [148]:


stock_table = db.Table('Stock', metadata,
              db.Column('id_stock', db.Integer(),primary_key=True),
              db.Column('nom_ingredient', db.String(),db.ForeignKey("Ingredient.nom_ingredient"),nullable=False),
              db.Column('code_postal', db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
              db.Column('quantite', db.Integer(),default=0),
              )



In [149]:


item_table = db.Table('Item', metadata,
              db.Column('nom_item', db.String(),primary_key=True),
              db.Column('type', db.String(),nullable=False),
              db.Column('prix', db.Float(),default=0.00),
              )



In [150]:

carte_item_table = db.Table('CarteItem', metadata,
              db.Column('id_carte_item', db.Integer(),primary_key=True),
              db.Column('nom_item', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
              db.Column('pays', db.String(),db.ForeignKey("Pays.pays"),nullable=False),
              )

In [151]:
employes_table= db.Table ('Employes', metadata,
            db.Column('id_employes',db.Integer(),primary_key=True),
            db.Column('nom',db.String(),nullable=False),
            db.Column('role',db.String(),nullable=False),
            db.Column('id_superieur',db.Integer(),db.ForeignKey("Employes.id_employes")),
            db.Column('anciennete',db.Integer(),default=0),
            db.Column('compte_bancaire',db.Integer()),
            db.Column('note',db.Integer()),
            db.Column('adresse',db.String()),
            db.Column('code_postal',db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
            )

In [152]:
bulletin_paye = db.Table('BulletinPaye', metadata,
              db.Column('id_employes', db.Integer(),db.ForeignKey("Employes.id_employes"),primary_key=True),
              db.Column('date', db.String(),primary_key=True,nullable=False),
              db.Column('salaire', db.Float(),nullable=False),
               db.Column('poste', db.String(),nullable=False),
              )

In [153]:
recette_table = db.Table('Recette', metadata,
        db.Column('id_recette', db.Integer(),primary_key=True),
        db.Column('nom_item', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('nom_ingredient', db.String(),db.ForeignKey("Ingredient.nom_ingredient"),nullable=False),
        db.Column('quantite_ingredient', db.Integer(),default=0)
        )

In [154]:
menu = db.Table('Menu', metadata,
        db.Column('nom_menu', db.String(),primary_key=True),
        db.Column('plat', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('dessert', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('boisson', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
        db.Column('prix', db.Integer(),default=0.00)
        )

In [155]:
carte_menu_table = db.Table('CarteMenu', metadata,
              db.Column('id_carte_menu', db.Integer(),primary_key=True),
              db.Column('nom_menu', db.String(),db.ForeignKey("Item.nom_item"),nullable=False),
              db.Column('pays', db.String(),db.ForeignKey("Pays.pays"),nullable=False),
              )

In [156]:
table_addition = db.Table('Addition', metadata,
              db.Column('id_ticket', db.Integer(),primary_key=True),
              db.Column('heure', db.Integer(),nullable=False),
              db.Column('code_postal', db.String(),db.ForeignKey("Restaurant.code_postal"),nullable=False),
              db.Column('id_employes', db.String(),db.ForeignKey("Employes.id_employes"),nullable=False),
              db.Column('somme', db.REAL(),default=0),
              db.Column('moyen_paiment', db.String(),nullable=False),
              )

In [157]:
panier_item_table = db.Table('PanierItem', metadata,
              db.Column('id_ticket', db.Integer(),db.ForeignKey("Addition.id_ticket"),primary_key=True),
              db.Column('nom_item', db.Integer(),db.ForeignKey("Item.nom_item"),primary_key=True),
              db.Column('quantite_item', db.Integer(),default=0),
              )

In [158]:
panier_menu_table = db.Table('PanierMenu', metadata,
              db.Column('id_panier_menu', db.Integer(), primary_key=True),
              db.Column('id_ticket', db.Integer(),db.ForeignKey("Addition.id_ticket"),nullable=False),
              db.Column('nom_menu', db.Integer(),db.ForeignKey("Menu.nom_menu"),nullable=False),
              db.Column('quantite_menu', db.Integer(),default=0),
              )

In [159]:
metadata.create_all(engine)

In [160]:
type_liste = ["boisson", "dessert", "plat"]
with engine.begin() as connection:
    for _ in range(30):
      insert_stmt = item_table.insert().values(
        nom_item = faker.unique.dish(),
        type = random.choice(type_liste),
        prix = round(random.uniform(2, 10),2),

      )
      connection.execute(insert_stmt)

In [161]:

with engine.begin() as connection:
    for _ in range(40):
      insert_stmt = ingredient_table.insert().values(
          nom_ingredient = faker.unique.ingredient(),
          prix_achat = round(random.uniform(0.2,8),2)

      )
      connection.execute(insert_stmt)

In [162]:
with engine.begin() as connection:
    item_pk = connection.execute(db.select([item_table.c.nom_item])).fetchall()
with engine.begin() as connection:
    ingredient_pk = connection.execute(db.select([ingredient_table.c.nom_ingredient])).fetchall()


with engine.begin() as connection:
    for i in range(len(item_pk)):
        ingredient_selectionne = []
        for nb_ingredient in range(random.randrange(1,6)):
            ingre = random.choice([ing for ing in ingredient_pk if ing not in ingredient_selectionne])[0]
            insert_stmt = recette_table.insert().values(
                
                nom_item = item_pk[i][0],
                nom_ingredient = ingre,
                
                quantite_ingredient = random.randrange(1,5),
            )
            ingredient_selectionne.append(ingre)
            
            connection.execute(insert_stmt)

In [163]:
with engine.begin() as connection:
    item_plat_pk = connection.execute(db.select([item_table.c.nom_item]).where(item_table.c.type=="plat")).fetchall()
with engine.begin() as connection:
    item_dessert_pk = connection.execute(db.select([item_table.c.nom_item]).where(item_table.c.type =="dessert")).fetchall()
with engine.begin() as connection:
    item_boisson_pk = connection.execute(db.select([item_table.c.nom_item]).where(item_table.c.type=="boisson")).fetchall()

with engine.begin() as connection:
    for _ in range(15):
      insert_stmt = menu_table.insert().values(
          nom_menu = faker.unique.company(),
          plat = random.choice(item_plat_pk)[0],
          dessert = random.choice(item_dessert_pk)[0],
          boisson = random.choice(item_boisson_pk)[0],
          prix = round(random.uniform(8,15),2)

      )
      connection.execute(insert_stmt)

In [164]:
with engine.begin() as connection:
    for _ in range(15):
      insert_stmt = menu_table.insert().values(
          nom_menu = faker.unique.company(),
          plat = random.choice(item_plat_pk)[0],
          dessert = random.choice(item_dessert_pk)[0],
          boisson = random.choice(item_boisson_pk)[0],
          prix = round(random.uniform(8,15),2)

      )
      connection.execute(insert_stmt)